# "Power" calculations with Bayesian ($\epsilon, \delta$) - PACMAN

By Roberto-Rafael Maura-Rivero 

This notebook has the goal of helping applied researchers get a sense of how the power calculations would translate from using an RCT to using the ($\epsilon, \delta$) - PACMAN Bandit algorithm. 

First of all, let's revisit what is the definition of Power: 

In the simplest escenario, where we care about $\theta$ being the ATE of a particular policy, and we have a very specific alternative hypothesis in mind (e.g. $H_A: \theta = 0.3$), power is defined as 1 - probability of type 2 error 

$\text{power} = 1-\beta = P(\text{reject } H_0|H_A: \theta = 0.3)$

In our paper, we suggest to deviate from the RCT and instead consider multiple treatments simultaneously over multiple experiments. The intuition is that, instead of organizing 3 RCTs to compare 3 different treatments, we sequentially run 3 experiments, with 8 treatments, then 4 and then 2. This way, we can compare 8 treatments in total, instead of 3.

(explain bandit algorithm here)

But if we are using bandits, what should we consider as "power"? It is not immediately obvious which object we should care about, but an intuitive suggestion is the following: 

You consider 8 bandit arms, that is 7 treatments and 1 control. 

Consider a very specific hypothesis on the ATE of multiple treatments 

e.g. 

$H: \\ \theta_0 = 0, \\ \theta_1 = 0.1,\\ \theta_2 = 0.2,\\ \theta_3 = 0.3,\\ \theta_4 = 0.4,\\ \theta_5 = 0.5,\\ \theta_6 = 0.6,\\ \theta_7 = 0.7$ 

Now, we can calculate the probability of "type 2 error" as the probability of returning the control as the best treatment.

"bandit power" = $Pr$( algorithm ($\epsilon, \delta$)-PACMAN returns control as best treatment | $H$)

In [13]:
!pip install numpy
!pip install tqdm



In [14]:
import numpy as np
from tqdm import trange



In [ ]:
# Necessary functions
def sample_complexity(epsilon_L, delta_L, alpha, beta, n_simulations=1000, n_samples=1000):
    n_treatments = len(alpha)
    MAX_N = 100

    left = 1
    right = MAX_N
    while left <= right:
        n = (left + right) // 2

        simulation_with_success = 0
        for _ in range(n_simulations):
            thetas = np.random.beta(alpha, beta, size=n_treatments)
            highest_theta = np.max(thetas)
            is_epsilon_optimal = np.abs(thetas - highest_theta) <= epsilon_L

            sum_Y = np.random.binomial(n, thetas)
            updated_alpha = alpha + sum_Y
            updated_beta = beta + n - sum_Y

            bool_survived_treatments = bool_choice_rule(updated_alpha, updated_beta, epsilon_L, n_samples)

            success = is_epsilon_optimal * bool_survived_treatments
            if np.sum(success) > 0:
                simulation_with_success += 1

        percentage_success = simulation_with_success / n_simulations

        if percentage_success > 1 - delta_L:
            right = n - 1
        else:
            left = n + 1

    return left

def bool_choice_rule(alpha, beta, epsilon_L, n_samples):
    n_treatments = len(alpha)
    thetas = np.random.beta(alpha[:, np.newaxis], beta[:, np.newaxis], size=(n_treatments, n_samples))
    highest_theta = np.max(thetas, axis=0)
    is_epsilon_optimal = np.abs(thetas - highest_theta) <= epsilon_L
    pr_epsilon_optimal = np.sum(is_epsilon_optimal, axis=1) / n_samples

    sorted_indices = np.argsort(pr_epsilon_optimal)[::-1]
    top_half_indices = sorted_indices[:n_treatments // 2]
    bool_survived_treatments = np.zeros(n_treatments)
    bool_survived_treatments[top_half_indices] = 1

    return bool_survived_treatments



n = 45


In [ ]:

epsilon = 0.05
epsilon_L = epsilon/3
delta = 0.05
delta_L = delta/3

# HYPOTHESIS of the average outcome  of each treatment i.e. E(Y|T_i)
theta = np.zeros(8)
theta[0] = 0.5
theta[1] = 0.55
theta[2] = 0.55
theta[3] = 0.55
theta[4] = 0.60
theta[5] = 0.60
theta[6] = 0.60
theta[7] = 0.65

print ("the ATE of your treatments are: ")
for i in np.arange(1,8):
    print ("treatment " + str(i) + " : " + str(round(theta[i]-theta[0], 2))) 

the ATE of your treatments are: 
treatment 1 : 0.05
treatment 2 : 0.05
treatment 3 : 0.05
treatment 4 : 0.1
treatment 5 : 0.1
treatment 6 : 0.1
treatment 7 : 0.15


In [ ]:
# prior is a uniform distribution for all treatments,i.e. distribution B(1,1)

# parameters of the prior beta distributions
alphas = np.zeros(8)
betas = np.zeros(8)

for i in range(8): 
    alphas[i] = 1
    betas[i] = 1

In [15]:

# SIMULATIONS
n_sim = 1000
n_remaining_treatments = 8
remaining_treatments = np.arange(8)

counter_type_2_error = 0
history_complexity = np.zeros(n_sim)
for i in trange(n_sim):
    # amount of people used in this simulation
    complexity_simulation = 0
    for village in range(3):
        # data recolection

        # number of people allocated to each treatment
        n_treatment = sample_complexity(epsilon_L=epsilon_L,
                    delta_L=delta_L,
                    alpha=alphas,
                    beta=betas,
                    n_simulations=10000)
        complexity_simulation += n_treatment
        
        # sample each treatment n_treatment times from a binomial
        # distribution with probability theta_i
        for i in range(8):
            data_treatment_i = np.random.binomial(n_treatment, theta[i])

        # update prior 
        for i in range(8):
            alphas[i] += data_treatment_i
            betas[i] += n_treatment - data_treatment_i

        # choose which treatments survive. 
        # to do that, we first calculate the probability of being epsilon optimal 

        # update remaining treatments
        n_remaining_treatments = len(remaining_treatments)
        if n_remaining_treatments == 1:

            # if there is only one treatment left, it is the best one
            # and we can stop the simulation
            break

    history_complexity[i] = complexity_simulation

    if 0 in remaining_treatments: 
        counter_type_2_error += 1


print("epsilon is: " + str(epsilon))
print("delta is: " + str(delta))

# tell me the average complexity of the simulations and relevant quantiles
print("the average complexity of the simulations is: " + str(np.mean(history_complexity)))
print("the 10th percentile of the complexity is: " + str(np.percentile(history_complexity, 10)))
print("the 90th percentile of the complexity is: " + str(np.percentile(history_complexity, 90)))

print ("the probability of type 2 error is: " + str(counter_type_2_error / n_sim))
print ("The \"bandit power\" is " + str(1 - counter_type_2_error / n_sim))

  0%|          | 1/1000 [02:27<41:03:49, 147.98s/it]


KeyboardInterrupt: 

In [16]:
import numpy as np
from tqdm import trange

# SIMULATIONS
n_sim = 1000
n_remaining_treatments = 8
remaining_treatments = np.arange(8)

counter_type_2_error = 0
history_complexity = np.zeros(n_sim)

for i in trange(n_sim):
    # amount of people used in this simulation
    complexity_simulation = 0
    alphas = np.ones(8)
    betas = np.ones(8)
    theta = np.random.rand(8)  # Random theta values for demonstration purposes
    epsilon_L = 0.01
    delta_L = 0.01

    for village in range(3):
        # data recolection
        n_treatment = sample_complexity(epsilon_L=epsilon_L,
                                        delta_L=delta_L,
                                        alpha=alphas,
                                        beta=betas,
                                        n_simulations=10000)
        complexity_simulation += n_treatment

        # sample each treatment n_treatment times from a binomial
        data_treatments = np.random.binomial(n_treatment, theta)

        # update prior
        alphas += data_treatments
        betas += n_treatment - data_treatments

        # update remaining treatments
        n_remaining_treatments = len(remaining_treatments)
        if n_remaining_treatments == 1:
            # if there is only one treatment left, it is the best one
            # and we can stop the simulation
            break

    history_complexity[i] = complexity_simulation

    if 0 in remaining_treatments:
        counter_type_2_error += 1

print("epsilon is: " + str(epsilon_L))
print("delta is: " + str(delta_L))

# tell me the average complexity of the simulations and relevant quantiles
print("the average complexity of the simulations is: " + str(np.mean(history_complexity)))
print("the 10th percentile of the complexity is: " + str(np.percentile(history_complexity, 10)))
print("the 90th percentile of the complexity is: " + str(np.percentile(history_complexity, 90)))

print("the probability of type 2 error is: " + str(counter_type_2_error / n_sim))
print("The \"bandit power\" is " + str(1 - counter_type_2_error / n_sim))

# it takes 3 hours to calculate 50 simulations

  5%|▌         | 50/1000 [2:46:11<52:37:42, 199.43s/it] 


KeyboardInterrupt: 

In [ ]:
# Now you get a graph with the bandit power of the test for each sample size

After experimenting with this you might say: 
Question:
Hey! I tried something like 

$H: \\ \theta_0 = 0, \\ \theta_1 = 0.1,\\ \theta_2 = 0.2,\\ \theta_3 = 0.3,\\ \theta_4 = 0.4,\\ \theta_5 = 0.5,\\ \theta_6 = 0.6,\\ \theta_7 = 0.7$ 

Yet the badit power is quite bad and the epsilon delta is quite good. How is that possible?

Answer:
Well, you see, the ($\epsilon, \delta$) depend on your prior believes about the treatment effects. In this notebook, we assumed a uniform prior, but you can change it to whatever you want. 

And, in fact, if you believe the effect of the treatment is fairly small, then you should reflect that in your prior. 

In the next cells we will show you how to do that. 